In [ ]:
import os
import csv
import math
from random import shuffle, choice
from pathlib import Path

# global movies list
movies = {}

# Define base directory path
base_dir = Path(r'C:\Users\surig\Desktop\Fuzzy-1\Movie_recommend\ml-latest-small')

# Function to read CSV files
def readCSV(fn):
    lis = []
    file_path = base_dir / fn  # Full path to the file
    with open(file_path, newline='', encoding='utf-8') as f:
        reader = csv.reader(f)
        for row in reader:
            lis.append(row)
    return lis[1:]

# Function that uses a list of id-movie combos and makes it into a global dict
def createMovieDict():
    movieList = [x[:2] for x in readCSV('movies.csv')]  # Using the correct path

    for i in movieList:
        # Remove the year from the title
        m = " ".join(i[1].split(" ")[:-1])
        movies[i[0]] = m.lower()

# Function to create a dictionary of users
def parseUsers():
    # Get the user ratings
    userRatings = readCSV('ratings.csv')  # Using the correct path

    # Put them into a dictionary
    users = {}

    for i in userRatings:
        u = i[0]  # User id
        m = movies[i[1]]  # Movie name
        r = float(i[2]) / 5  # rating limited to range of [0 .. 1]

        # If the user already has a section in the dict, user that
        if u in users:
            users[u][m] = r
        else:
            users[u] = {m: r}

    return users

# Function to read in ratings from standard input
def getInput():
    print("Enter the name of a movie followed by a space then a rating from 0-5 stars, decimals are allowed")
    print("Example: Toy Story 4.5")
    print("The movie title must follow exactly what is in the 'movies.csv' spreadsheet")
    print("Enter as many as you would like and then type 'quit' to quit")

    userData = {}

    userIn = input()

    while userIn.lower() != "quit":    
        try:
            userIn = userIn.split(" ")

            m = " ".join(userIn[:-1]).lower()
            r = float(userIn[-1]) / 5

            # Error checking
            if r > 1:
                print("Rating too high")
                raise Exception
            elif r < 0:
                print("Rating too low")
                raise Exception

            if m not in list(movies.values()):
                print("Movie not in list. Please enter another one")
                raise Exception

            # Add value to dictionary
            userData[m] = r
        except:
            print("You did something wrong. Please try again")
        
        userIn = input()

    print("Here is your membership set:")
    print(userData)
    return userData

# Function to calculate the unit vector of a vector
def unit_vector(vector):
    return vector / np.linalg.norm(vector)

# Function to calculate the angle between two vectors
def angle_between(v1, v2):
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))

# Function to compare similarity between two users
def sim(a, b):
    aMovies = set(list(a.keys()))
    bMovies = set(list(b.keys()))

    both = list(aMovies.intersection(bMovies))

    if len(both) <= 0:
        return 0
    elif len(both) == 1:
        return 1 - abs(a[both[0]] - b[both[0]])

    if aMovies.issubset(bMovies):
        return 0

    aFuzz = []
    bFuzz = []

    for m in both:
        aFuzz.append(a[m])
        bFuzz.append(b[m])

    return math.cos(angle_between(aFuzz, bFuzz))

# Function to compare the list of ratings to the user's ratings
def compareRatings(ratings, user):
    simVal = []

    for k, v in ratings.items():
        simVal.append([k, sim(v, user)])

    simVal.sort(key=lambda x: x[1])
    simVal.reverse()

    return simVal[:25]

# Function to get a list of movies that a user has rated
def getRatedMovies(ratingList, id):
    return ratingList[id].keys()

# Function that suggests movies based on similarity
def suggestMovies(ratingList, matches, user):
    watchedMovies = set(list(user.keys()))

    suggested = []

    for i in matches:
        suggested += getRatedMovies(ratingList ,i[0])

    suggested = set(suggested)
    suggested = list(suggested.difference(watchedMovies))

    shuffle(suggested)

    print("Below are some suggested movies for you based on your ratings:")
    for i in range(15):
        print("\t" + suggested[i])

# Main function
def main():
    createMovieDict()

    ratingList = parseUsers()

    print("Press 'y' to use random ratings, or press anything else to create your own movie ratings")
    if input() == "y":
        print("Here is the membership set you will be using:")
        randKey = choice(list(ratingList.keys()))
        userRating = ratingList[randKey]
        del ratingList[randKey]
        print(userRating)
    else:
        userRating = getInput()

    highestMatches = compareRatings(ratingList, userRating)

    suggestMovies(ratingList, highestMatches, userRating)

main()


Press 'y' to use random ratings, or press anything else to create your own movie ratings
Enter the name of a movie followed by a space then a rating from 0-5 stars, decimals are allowed
Example: Toy Story 4.5
The movie title must follow exactly what is in the 'movies.csv' spreadsheet
Enter as many as you would like and then type 'quit' to quit
You did something wrong. Please try again
You did something wrong. Please try again
